In [1]:
pip install openai==0.28

  Using cached aiohttp-3.9.3-cp39-cp39-win_amd64.whl.metadata (7.6 kB)
  Using cached multidict-6.0.5-cp39-cp39-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp39-cp39-win_amd64.whl.metadata (32 kB)
   ---------------------------------------- 76.5/76.5 kB 2.1 MB/s eta 0:00:00
   --------------------------------------- 366.0/366.0 kB 11.5 MB/s eta 0:00:00
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
   ---------------------------------------- 50.7/50.7 kB ? eta 0:00:00
Using cached multidict-6.0.5-cp39-cp39-win_amd64.whl (28 kB)
   ---------------------------------------- 76.9/76.9 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from tqdm import tqdm
import openai
import time
import numpy as np

In [12]:
pwd

'c:\\Users\\r2com\\Documents\\GitHub\\Resume'

In [48]:
df = pd.read_csv('C:/Users/r2com/Documents/GitHub/Resume/jobkorea.csv')

In [49]:
question_df = df.copy()

In [50]:
question_df.head()

,organization_name,role,question,answer,text_label
0,금호석유화학(주),2019년 하반기 신입 품질관리자,Challenge A. 당신이 지원한 직무와 관련하여 보유한 직무역량을 서술하십시오.,고분자 관련 지식과 분석 역량을 길렀습니다.좋은점 1\n글자수 26자\n44Byte,1
1,금호석유화학(주),2019년 하반기 신입 품질관리자,"해당 직무역량을 향상시키기 위해 노력했던 경험(학업,교육,자격증,직무경험,기타경험 ...","고분자 합성, 물성 과목을 수강하여 중합 및 성형 방법, 점탄성 특성, 분석 방법을...",1
2,금호석유화학(주),2019년 하반기 신입 품질관리자,"금호석유화학그룹에 입사할 경우, 본인이 현재 보유하거나, 앞으로 보유할 직무역량을 ...",제가 지원한 직무는 제품 및 원부재료를 관리하여 품질경쟁력을 높이고 제품의 성분과 ...,1
3,금호석유화학(주),2019년 하반기 신입 품질관리자,"Change A. 살아오는 동안 자신에게 생긴 변화 중 인생의 전환점이나, 가치관의...",OO장애인의 불편함을 개선한 보조기구를 제작하여 캡스톤디자인 경진대회 대상을 받은 ...,1
4,금호석유화학(주),2019년 하반기 신입 품질관리자,B. 어떤 이유로 변화 혹은 개선해야 한다고 생각했는지 서술하십시오.,동아리 봉사활동 중 OO 장애인분이 탁구 운동 시 붕대로 번거롭게 손을 고정하는 것...,1


In [51]:
question_df['Text'] = question_df['organization_name'] + ',' + question_df['role'] + ',' + question_df['question'] 

In [52]:
question_df.shape

(30743, 6)

In [53]:
question_df = pd.DataFrame(question_df[:5000]['Text'])

In [54]:
question_df

,Text
0,"금호석유화학(주),2019년 하반기 신입 품질관리자,Challenge A. 당신이 ..."
1,"금호석유화학(주),2019년 하반기 신입 품질관리자,해당 직무역량을 향상시키기 위해..."
2,"금호석유화학(주),2019년 하반기 신입 품질관리자,금호석유화학그룹에 입사할 경우,..."
3,"금호석유화학(주),2019년 하반기 신입 품질관리자,Change A. 살아오는 동안..."
4,"금호석유화학(주),2019년 하반기 신입 품질관리자,B. 어떤 이유로 변화 혹은 개..."
...,...
4995,"현대글로비스(주),2022년 하반기 신입 해외영업,회사를 선택할 때, 지원자님만의 ..."
4996,"현대글로비스(주),2022년 하반기 신입 해외영업,지원하신 부문과 관련하여 당사에서..."
4997,"현대글로비스(주),2022년 하반기 신입 해외영업,아래의 질문 중 하나를 선택하여 ..."
4998,"신용협동조합중앙회,2019년 하반기 신입 금융영업,본인에 대한 소개 및 본인의 강점..."


In [55]:
# openAI에서 받아온 API 키
openai.api_key = ""


# 'answer' 열이 없으면 추가하기
if 'answer' not in question_df.columns:
    question_df['answer'] = np.nan  # 빈칸을 NaN으로 채움

# GPT 모델에게 부여할 역할 설정
system_role = '''당신은 자기소개서 작성 전문가입니다. 제가 문항을 알려드리면, 해당 문항에 대한 답변을 작성해주세요. 만약 Text 안에 글자수가 명시되어있지 않다면 가능하면 500자 기준으로 작성해주세요.'''

# 문항 개수만큼 반복문
for i in tqdm(range(len(question_df))):
    if pd.notna(question_df['answer'][i]):  # 이미 답변이 있는 경우 넘어감
        continue
    else:
        # 각 질문에 대해 답변 생성
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_role},
                {"role": "user", "content": question_df['Text'][i]}
            ],
            #max_tokens=500,
            temperature=0.7
        )

        generated_text = completion['choices'][0]['message']['content']

        question_df.loc[i, 'answer'] = generated_text.strip()  # 생성된 답변 저장

    # API 호출 제한을 막기 위한 0.5초 딜레이
    time.sleep(0.5)

# 결과를 파일에 저장
question_df.to_csv('C:/Users/r2com/Documents/GitHub/Resume/wrong_answer.csv', index=False)


 12%|█▏        | 617/5000 [1:22:10<9:43:41,  7.99s/it] 


APIError: Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_43f86f513584a3be865b5b867557bbcf in your message.) {
  "error": {
    "message": "Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_43f86f513584a3be865b5b867557bbcf in your message.)",
    "type": "server_error",
    "param": null,
    "code": "invalid_model_output"
  }
}
 500 {'error': {'message': 'Failed to create completion as the model generate

In [74]:
anomal = question_df[:617]

In [75]:
anomal['text_label'] = 0


C:\Users\r2com\AppData\Local\Temp\ipykernel_12780\3685218950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomal['text_label'] = 0


In [77]:
anomal.to_csv('anomal.csv', index=False)

In [80]:
anomal

,Text,answer,text_label
0,"금호석유화학(주),2019년 하반기 신입 품질관리자,Challenge A. 당신이 ...",저는 공학 학위를 가지고 있으며 품질 관리에 대한 이론적인 지식과 실무 경험이 있습...,0
1,"금호석유화학(주),2019년 하반기 신입 품질관리자,해당 직무역량을 향상시키기 위해...","저는 품질관리자로서의 역량을 향상시키기 위해 다양한 경험을 쌓았습니다. 먼저, 대학...",0
2,"금호석유화학(주),2019년 하반기 신입 품질관리자,금호석유화학그룹에 입사할 경우,...",저는 품질관리 업무에 대한 깊은 이해와 열정을 가지고 있습니다. 제가 금호석유화학그...,0
3,"금호석유화학(주),2019년 하반기 신입 품질관리자,Change A. 살아오는 동안...",저에게 있었던 인생의 전환점은 대학 졸업 후 처음으로 취업을 준비하던 시기였습니다....,0
4,"금호석유화학(주),2019년 하반기 신입 품질관리자,B. 어떤 이유로 변화 혹은 개...",금호석유화학(주)에서 품질관리자로 근무하면서 변화와 개선이 필요하다고 느낀 이유는 ...,0
...,...,...,...
612,"H그린파워㈜,2021년 상반기 신입 품질관리자,지원 직무에 대해 자신이 적합하다고 ...",저는 H그린파워㈜의 신입 품질관리자 지원 직무에 적합하다고 자부합니다. 제가 이 직...,0
613,"H그린파워㈜,2021년 상반기 신입 품질관리자,에이치엘그린파워에 입사 후 성장방향/...",에이치엘그린파워에 입사한 지 6개월이 지난 신입 품질관리자입니다. 이 기간 동안 많...,0
614,"H그린파워㈜,2021년 상반기 신입 품질관리자,타인과 협동하여 문제를 해결하는 과정...",갈등 상황에서 제가 취한 조치는 먼저 상황을 잘 이해하고 상대방의 의견을 경청하는 ...,0
615,"삼성SDI(주),2015년 상반기 신입 생산직종사자,삼성취업을 선택한 이유와 입사 ...",삼성SDI(주)에 지원한 이유는 그들의 혁신적이고 글로벌한 기업문화에 큰 매력을 느...,0


In [85]:
anomal['char_count'] = anomal['answer'].apply(len)

C:\Users\r2com\AppData\Local\Temp\ipykernel_12780\1401108023.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anomal['char_count'] = anomal['answer'].apply(len)


In [94]:
anomal['char_count'].describe()

count    614.000000
mean     392.330619
std      135.676132
min       43.000000
25%      305.000000
50%      366.000000
75%      451.750000
max      980.000000
Name: char_count, dtype: float64

In [93]:
# 'char_count' 값이 1000을 넘어가는 행들을 제거합니다.
anomal = anomal[anomal['char_count'] <= 1000]

In [96]:
anomal.to_csv('anomal.csv', index=False)